# Initilization

In [0]:
!mkdir -p models
!wget "https://drive.google.com/uc?id=1E95HNEYQI1R-UTuYwJOycrYDJxFxiICC&export=download" -O songs.p
!pip install tensorflow-gpu==1.14

# Dataset Preparation

In [0]:
%pylab inline
import pickle
import pandas as pd
import keras
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential, load_model
from keras.layers import LSTM, Bidirectional, Dropout, Dense, Activation
from keras.callbacks import ModelCheckpoint, History

# implementation based on https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
def generate_dataset():
  """datas preprocessing based on a list of song objects, each song object contains song's name and a sequence of notes"""
  songs = pickle.load(open("songs.p", "rb"))

  # get a list of all notes in all songs
  notes = []
  for song in songs:
    notes += song["notes"]

  # n_vocab is the number of unique netes
  n_vocab = len(set(notes))

  # length of input sequence to LSTM network 
  sequence_length = 100
  # get all pitch names
  pitchnames = sorted(set(item for item in notes))
  # create a dictionary to map pitches to integers
  note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
  int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

  network_input = []
  network_output = []
  # create input sequences and the corresponding outputs
  for song in songs:
    print("Loading", song["name"])
    notes = song["notes"]

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i : i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

  n_patterns = len(network_input)
  # reshape the input into a format compatible with LSTM layers
  network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
  # normalize input
  network_input = network_input / float(n_vocab)
  network_output = keras.utils.to_categorical(network_output)

  return (n_vocab, int_to_note, network_input, network_output)

# Create Network

In [0]:
def create_network(network_input, n_vocab):
    """create network structure"""
    model = Sequential()
    model.add(LSTM(128,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(256)))
    model.add(Dense(128))
    model.add(Dropout(0.5))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

# Train Network

In [0]:
def train_network():
  """train the network"""
  (n_vocab, _, network_input, network_output) = generate_dataset()

  # get model structure
  model = create_network(network_input, n_vocab)
  model.summary()

  # callbacks
  history = History()
  filepath = 'models/model-{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5'
  checkpoint_cb = ModelCheckpoint(
      filepath, monitor='loss', verbose=0, save_best_only=True, mode='min'
  )

  # start training the model
  n_epoch = 30
  model.fit(network_input, network_output, epochs=n_epoch, batch_size=64, validation_split=0.2,
            callbacks=[history, checkpoint_cb])
  model.save('music_generate_model.h5')

  # Plot the model losses
  pd.DataFrame(history.history).plot()
  plt.savefig('network_loss_per_epoch.png', transparent=True)
  plt.close()

train_network()

# Model Prediction

In [0]:
# Model predicion's implementation based on https://github.com/corynguyen19/midi-lstm-gan

def generate_notes(model):
    """ Generate notes from the neural network based on a sequence of notes """
    (n_vocab, int_to_note, network_input, _) = generate_dataset()

    # pick a random sequence from the input as a starting point for the prediction  
    start = np.random.randint(0, len(network_input) - 1)

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for i in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        # random choose from a distribution
        index = np.random.choice(n_vocab, 1, p=prediction[0])[0]
        # index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        print(result)
        
        pattern = np.append(pattern, index)
        pattern = pattern[1 : len(pattern)]

    return prediction_output
  

def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# model = load_model('/content/models/model-20-1.53-6.50.hdf5')
# prediction_output = generate_notes(model)
# create_midi(prediction_output, 'test')

#Download files

In [0]:
from google.colab import files
from google.colab import drive

# files.download('test.mid')
drive.mount('/content/gdrive',force_remount=True)

In [0]:
!zip models.zip models/*
!cp models.zip '/content/gdrive/My Drive/'